In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_validate

from prepare_datasets_HCSC import load_data_HCSC
from prepare_datasets_ADNI import load_data_ADNI, process_descriptive_variables
from clusters_description import label_A, label_T, label_N

In [2]:
sns.set(style='whitegrid', font='Arial', font_scale=1.)

## Define datasets/cohorts
### Discovery dataset: HCSC cohort

In [3]:
file_dis = 'data/HCSC/BaseLCR_24julio22conATNparaLaura.sav'
des_dis, csf_dis, cog_dis = load_data_HCSC(file_dis)
des_dis['DxclinBreve'].replace({'EA GDS3':'LMCI',
                                'EApreMCI':'EMCI',
                                'Control/QSM':'SMC',
                                'Otros':'MCI-NN'}, inplace=True)
des_dis = des_dis.drop([510673, 1175866, 1106441, 622301, 2378737])
csf_dis = csf_dis.drop([510673, 1175866, 1106441, 622301, 2378737])
x_dis = csf_dis[['RatioR', 'TAUtotal', 'pTAU']].dropna()
print('Number of subjects:', x_dis.shape[0])

# Scaling data
scaler = StandardScaler()
scaler = scaler.fit(x_dis)
x_dis_scaled  = scaler.transform(x_dis)

Number of subjects: 165


/home/laura/anaconda3/envs/biomarkers/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


### Validation dataset: ADNI cohort

In [7]:
# Validation cohort
val_file = 'data/ADNI/UPENNBIOMK10_07_29_19_19Jun2023.csv'
csf_val = pd.read_csv(val_file, index_col='RID')
csf_val = csf_val.loc[csf_val['VISCODE2'] == 'bl']
csf_val = csf_val[['ABETA42', 'ABETA40', 'TAU', 'PTAU']].dropna()
csf_val['AB4240'] = csf_val['ABETA42'] / csf_val['ABETA40']

des_val = pd.read_csv('data/ADNI/descriptive_variables.csv', low_memory=False, index_col='RID')
des_val = des_val.drop(index=des_val.loc[des_val['DX_bl'] == 'AD'].index) # Delete dementia patients

x_val = pd.concat([csf_val[['AB4240', 'TAU', 'PTAU']], des_val['DX_bl']], axis=1, join='inner')
x_val.drop(columns=['DX_bl'], inplace=True)
print('Number of subjects:', x_val.shape[0])

# Scaling data
scaler = StandardScaler()
scaler = scaler.fit(x_val)
x_val_scaled  = scaler.transform(x_val)

Number of subjects: 174


### Define optimal number of clusters

In [9]:
for n in range(2, 11):

    km = KMeans(n_clusters=n, max_iter=1000, random_state=42, n_init=100)
    y_km = km.fit_predict(x_dis_scaled)

    si = round(silhouette_score(x_dis_scaled, y_km, metric='euclidean'), 4)
    print(n, si) 

   # optimal number of clusters: n = 3

2 0.5297
3 0.5527
4 0.5279
5 0.496
6 0.4511
7 0.3772
8 0.3795
9 0.377
10 0.3735


### Clustering model
Train clustering model on dicovery cohort and obtain clusters in discovery and validation cohorts.

In [10]:
km = KMeans(n_clusters=3, max_iter=1000, random_state=42, n_init=100)
y_train = km.fit_predict(x_dis_scaled)
y_test  = km.predict(x_val_scaled)

si_train = round(silhouette_score(x_dis_scaled, y_train, metric='euclidean'), 4)
si_test  = round(silhouette_score(x_val_scaled, y_test, metric='euclidean'), 4)

print('Discovery (train):', si_train)
print('Validation (test) ', si_test)

Discovery (train): 0.5527
Validation (test)  0.3847


### Validation of clusters
Train a simple classification model (Logistic Regression) using discovery cohort (HCSC dataset) and evaluate or test it using validation cohort (ADNI cohort).

In [14]:
model = LogisticRegression(solver='lbfgs', random_state=42, max_iter=1000)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_results = cross_validate(model, x_dis_scaled, y_train, cv=kfold,
                            scoring=['accuracy','precision_weighted',
                                     'recall_weighted','f1_weighted',
                                     'matthews_corrcoef'])
clf = model.fit(x_dis_scaled, y_train)
cv_results_df = pd.DataFrame(cv_results)
cv_results_df

,fit_time,score_time,test_accuracy,test_precision_weighted,test_recall_weighted,test_f1_weighted,test_matthews_corrcoef
0,0.006631,0.003604,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.003866,0.002170,1.000000,1.000000,1.000000,1.000000,1.000000
2,0.002879,0.001812,0.941176,0.949580,0.941176,0.941478,0.911616
3,0.002618,0.002016,1.000000,1.000000,1.000000,1.000000,1.000000
4,0.002967,0.001843,1.000000,1.000000,1.000000,1.000000,1.000000
5,0.002760,0.001824,0.937500,0.945312,0.937500,0.929167,0.899388
6,0.003032,0.001800,0.937500,0.945312,0.937500,0.929167,0.899388
7,0.002880,0.001863,1.000000,1.000000,1.000000,1.000000,1.000000
8,0.002853,0.001853,1.000000,1.000000,1.000000,1.000000,1.000000
9,0.002959,0.001771,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
y_pred = clf.predict(x_val_scaled)

print(classification_report(y_pred=y_pred, y_true=y_test))
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print()

acc = metrics.accuracy_score(y_pred=y_pred, y_true=y_test)
pre = metrics.precision_score(y_pred=y_pred, y_true=y_test, average='weighted')
rec = metrics.recall_score(y_pred=y_pred, y_true=y_test, average='weighted')
f1  = metrics.f1_score(y_pred=y_pred, y_true=y_test, average='weighted')
mcc = metrics.matthews_corrcoef(y_pred=y_pred, y_true=y_test)

print('Accuracy: ', round(acc, 4))
print('Precision:', round(pre, 4))
print('Recall   :', round(rec, 4))
print('F1-Score: ', round(f1, 4))
print('MCC:      ', round(mcc, 4))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93        86
           1       0.87      0.90      0.88        67
           2       0.95      0.86      0.90        21

    accuracy                           0.91       174
   macro avg       0.92      0.89      0.90       174
weighted avg       0.91      0.91      0.91       174

[[80  6  0]
 [ 6 60  1]
 [ 0  3 18]]

Accuracy:  0.908
Precision: 0.9089
Recall   : 0.908
F1-Score:  0.9081
MCC:       0.8443
